# Import related pacakages

In [1]:
import cobra
import sys
sys.path.append(r'./code/')
from cobrapy_ec_model_function import *

# Inputing files

In [2]:
# The genome-scale metabolic model for constructing the enzyme-constrained model
model_name = './data/iML1515.xml' 

# Reaction-kcat file.#s-1
# eg. AADDGT,49389.2889
#reaction_kcat_file = "./data/reaction_autopacmen_ori_noadj.csv"
#reaction_kcat_file = "./data/reaction_kappori.csv"
reaction_kcat_file = "./data/reaction_autopacmen_ori_adj.csv"

# Gene-abundance file. 
# eg. b0789,1.1
gene_abundance_file = "./data/gene_abundance.csv"

# Gene-molecular_weight file. 
# eg. b0001,thrL,2.13846
gene_molecular_weight_file = "./data/gene_molecular_weight.csv"


# Step1: Preprocessing of model

The reversible reactions in the GEM model are divided into two irreversible reactions. The input is iML1515 with 2712 reactions. The output is a model with 3375 irreversible reactions.

In [3]:
model = cobra.io.read_sbml_model(model_name)
convert_to_irreversible(model)
model

Name,iML1515
Memory address,0x01d0cca3a860
Number of metabolites,1877
Number of reactions,3375
Number of groups,38
Objective expression,1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


# Step2: Retrieving enzyme kinetics and proteomics data

The inputs are GEM model. The outputs are 'genes' and 'gpr_relationship' data in the iML1515.

In [4]:
[genes,gpr_relationship] = get_genes_and_gpr(model)

Get the molecular weight of the enzyme (MW) according to the file of all_reaction_GPR.csv, which obtained from the previous step (gpr_relationship, ./analysis/all_reaction_GPR.csv). We need to manually correct the error of the gene_reaction_rule of a small amount of reactions in iML1515 (See Supplementary Table S1 for details), and also need to manually get the subunit of each protein from EcoCyc.

In [5]:
# reaction-gene-subunit-MW file. 
# eg. ALATA_D2,D-alanine transaminase,b2551 or b0870,45.31659 or 36.49471 ,2 or 4 
reaction_gene_subunit_MW = "./data/reaction_gene_subunit_MW_old.csv"
reaction_mw = calculate_reaction_mw(reaction_gene_subunit_MW)

Calculate kcat/MW. The inputs are 'reaction_kcat' and 'reaction_MW' data for calculating the kcat/MW (When the reaction is catalyzed by several isozymes, the maximum is retained).

In [6]:
save_file="./analysis/reaction_kcat_mw.csv"
reaction_kcat_mw = calculate_reaction_kcat_mw_old(reaction_kcat_file, reaction_mw, save_file)

Calculate f. The input is 'genes' data, 'gene_abundance.csv' and 'gene_molecular_weight.csv'.

In [7]:
f = calculate_f(genes, gene_abundance_file, gene_molecular_weight_file)
f

0.4059986079578236

# Step3: Save enzyme concentration constraint model as json file.

In [8]:
model_file = './data/iML1515.xml' 
reaction_kcat_mw_file="./analysis/reaction_kcat_mw.csv"
json_output_file="./model/iML1515_irr_enz_constraint_kapp.json"
#The enzyme mass fraction 
f = 0.406
# The total protein fraction in cell.
ptot = 0.56 
# The approximated average saturation of enzyme.
#sigma = 0.5 
sigma = 1 #kapp data sigma is 1
# Lowerbound  of enzyme concentration constraint. 
lowerbound = 0   
upperbound = round(ptot * f * sigma, 3)

trans_model2enz_json_model(model_file, reaction_kcat_mw_file, f, ptot, sigma, lowerbound, upperbound, json_output_file)

In [9]:
json_model_path = "./model/iML1515_irr_enz_constraint_kapp.json"
fluxes_outfile = './analysis/ECMpy_kappori_solution_df_pfba.csv'
reaction_kcat_mw_file="./analysis/reaction_kcat_mw.csv"
enz_model=get_enzyme_constraint_model(json_model_path)
enz_model_pfba_solution = get_fluxes_detail_in_model(enz_model,fluxes_outfile,reaction_kcat_mw_file)
print(enz_model_pfba_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M'])

norm_model = cobra.io.json.load_json_model(json_model_path)
norm_model_pfba_solution = cobra.flux_analysis.pfba(norm_model)
norm_model_pfba_solution_df = norm_model_pfba_solution.to_frame()
norm_model_pfba_solution_df.to_csv('./analysis/Orimodel_solution_df_pfba.csv')
print(norm_model_pfba_solution_df.fluxes['BIOMASS_Ec_iML1515_core_75p37M'])

0.6453070777769297
0.8769972144269698


# Step4: Calibration parameters

Compare with C13 data

In [10]:
c13reaction_file = './data/C13reaction.csv' 
c13reaction = pd.read_csv(c13reaction_file, index_col=0)
c13reaction = list(c13reaction.index)

enz_model_pfba_solution_select = enz_model_pfba_solution[enz_model_pfba_solution['fluxes']>0]
enz_model_pfba_solution_select_id = list(enz_model_pfba_solution_select.index)
c13reaction_2_enz_model_diff=list(set(c13reaction).difference(set(enz_model_pfba_solution_select_id)))
print (c13reaction_2_enz_model_diff)

['PFK', 'ACKr_reverse', 'FBA', 'PTAr', 'ICL', 'PGI', 'SUCOAS_reverse', 'PYK', 'ME2', 'AKGDH', 'TPI', 'PDH']


Select calibration reaction accrording biomass diff

In [11]:
reaction_kcat_mw_file = "./analysis/reaction_kcat_mw.csv"
json_model_path = "./model/iML1515_irr_enz_constraint_kapp.json"
enzyme_amount = upperbound
percentage = 0.1#10%
reaction_biomass_outfile = './analysis/reaction_biomass.csv'
select_percentage = 0.8
df_reaction_select = select_calibration_reaction_by_biomass(reaction_kcat_mw_file, json_model_path, enzyme_amount, percentage, reaction_biomass_outfile, select_percentage)
df_reaction_select

,biomass_enz,biomass_orimodel,biomass_diff,biomass_diff_ratio
ATPS4rpp,0.438292,0.876997,0.438705,0.500235
KARA1_reverse,0.619167,0.876997,0.257830,0.293992


In [12]:
need_change_reaction=list(df_reaction_select.index)
reaction_kcat_mw_file = "./analysis/reaction_kcat_mw.csv"
reaction_kapp_change_file = "./analysis/reaction_kapp_change.csv"
kcat_data_colect_file="./analysis/reaction_max_df.csv"
change_reaction_list_round1=change_reaction_kcat_by_database_kapp(need_change_reaction,kcat_data_colect_file,reaction_kcat_mw_file,reaction_kapp_change_file)
print(change_reaction_list_round1)

['ATPS4rpp', 'KARA1_reverse']


In [13]:
model_file = './data/iML1515.xml' 
reaction_kcat_mw_file="./analysis/reaction_kapp_change.csv"
json_output_file="./model/iML1515_irr_enz_constraint_adj_round1.json"
#The enzyme mass fraction 
f = 0.406
# The total protein fraction in cell.
ptot = 0.56 
# The approximated average saturation of enzyme.
#sigma = 0.5 
sigma = 1#kcapp数据，饱和度为1
# Lowerbound  of enzyme concentration constraint. 
lowerbound = 0   
upperbound = round(ptot * f * sigma, 3)

trans_model2enz_json_model(model_file, reaction_kcat_mw_file, f, ptot, sigma, lowerbound, upperbound, json_output_file)

In [14]:
json_model_path = "./model/iML1515_irr_enz_constraint_adj_round1.json"
fluxes_outfile= './analysis/ECMpy_kapp_adj_round1_solution_df_pfba.csv'
reaction_kcat_mw_file="./analysis/reaction_kapp_change.csv"
enz_model=get_enzyme_constraint_model(json_model_path)
enz_model_pfba_solution = get_fluxes_detail_in_model(enz_model,fluxes_outfile,reaction_kcat_mw_file)
print(enz_model_pfba_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M'])

0.847804490949618


In [15]:
c13reaction_file = './data/C13reaction.csv' 
c13reaction = pd.read_csv(c13reaction_file, index_col=0)
c13reaction = list(c13reaction.index)

enz_model_pfba_solution_select = enz_model_pfba_solution[enz_model_pfba_solution['fluxes']>0]
enz_model_pfba_solution_select_id = list(enz_model_pfba_solution_select.index)
c13reaction_2_enz_model_diff=list(set(c13reaction).difference(set(enz_model_pfba_solution_select_id)))
print (c13reaction_2_enz_model_diff)

['ACKr_reverse', 'ICL', 'PYK', 'PGI', 'ME2', 'PTAr']


Calibration kcat according c13 reaction list

In [17]:
c13reaction_file = './data/C13reaction.csv' 
reaction_kcat_mw_file = "./analysis/reaction_kcat_mw.csv"
f = 0.406
ptot = 0.56 
sigma = 1 
enzyme_amount = round(ptot * f * sigma, 3)

c13reaction_selecet=select_calibration_reaction_by_c13(reaction_kcat_mw_file, c13reaction_file, enzyme_amount, 0.1, sigma)
print(c13reaction_selecet)

['PDH']


In [18]:
reaction_kcat_mw_file = "./analysis/reaction_kapp_change.csv"
reaction_kapp_change_file = "./analysis/reaction_kapp_change_c13.csv"
#c13reaction_selecet=['CS','ACONTa','ACONTb','ICDHyr','MALS', 'MDH', 'ICL', 'SUCOAS_reverse', 'SUCDi', 'AKGDH']
kcat_data_colect_file="./analysis/reaction_max_df.csv"
change_reaction_list_round1=change_reaction_kcat_by_database_kapp(c13reaction_selecet,kcat_data_colect_file,reaction_kcat_mw_file,reaction_kapp_change_file)
print(change_reaction_list_round1)

[]


In [19]:
model_file = './data/iML1515.xml' 
reaction_kcat_mw_file="./analysis/reaction_kapp_change_c13.csv"
json_output_file="./model/iML1515_irr_enz_constraint_adj_round2.json"
f = 0.406
ptot = 0.56 
sigma = 1
lowerbound = 0   
upperbound = round(ptot * f * sigma, 3)

trans_model2enz_json_model(model_file, reaction_kcat_mw_file, f, ptot, sigma, lowerbound, upperbound, json_output_file)

In [20]:
json_model_path = "./model/iML1515_irr_enz_constraint_adj_round2.json"
fluxes_outfile = './analysis/ECMpy_kapp_adj_round2_solution_df_pfba.csv'
reaction_kcat_mw_file="./analysis/reaction_kapp_change_c13.csv"
enz_model=get_enzyme_constraint_model(json_model_path)
enz_model_pfba_solution = get_fluxes_detail_in_model(enz_model,fluxes_outfile,reaction_kcat_mw_file)
print(enz_model_pfba_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M'])

0.847804490949618


In [21]:
c13reaction_file = './data/C13reaction.csv' 
c13reaction = pd.read_csv(c13reaction_file, index_col=0)
c13reaction = list(c13reaction.index)

enz_model_pfba_solution_select = enz_model_pfba_solution[enz_model_pfba_solution['fluxes']>0]
enz_model_pfba_solution_select_id = list(enz_model_pfba_solution_select.index)
c13reaction_2_enz_model_diff=list(set(c13reaction).difference(set(enz_model_pfba_solution_select_id)))
print (c13reaction_2_enz_model_diff)

['ACKr_reverse', 'ICL', 'PYK', 'PGI', 'ME2', 'PTAr']


Calibration kcat according Enzyme usage 

In [22]:
f = 0.406
ptot = 0.56 
sigma = 1
enz_total = round(ptot * f * sigma, 3)
reaction_flux_file = './analysis/ECMpy_kapp_adj_round2_solution_df_pfba.csv' 
reaction_kcat_mw_file = "./analysis/reaction_kapp_change_c13.csv"
reaction_enz_usage_file = './analysis/ECMpy_kapp_adj_round2_reaction_enz_usage_df.csv'

reaction_enz_usage_df = get_enzyme_usage(enz_total,reaction_flux_file,reaction_kcat_mw_file,reaction_enz_usage_file)
reaction_enz_usage_df.head(22)

,kcat_mw,flux,enz useage,enz ratio
PDH,132.494090,4.662050,0.035187,0.155008
NADH16pp,1406.082994,45.339488,0.032245,0.142050
PSERT,66.819799,1.410051,0.021102,0.092962
ATPS4rpp,5297.986256,77.851493,0.014695,0.064734
GND,2633.457211,18.251223,0.006931,0.030531
FBA,472.952582,3.178421,0.006720,0.029605
PAPSR2,33.787531,0.209816,0.006210,0.027356
KARA2,42.209426,0.246312,0.005835,0.025707
TKT2,1156.921394,5.593318,0.004835,0.021298
ENO,2525.688868,10.466476,0.004144,0.018256


In [23]:
select_reaction = list(reaction_enz_usage_df[reaction_enz_usage_df['enz ratio']>0.01].index)#more than 1%
print(select_reaction)
reaction_kcat_mw_file = "./analysis/reaction_kapp_change_c13.csv"
reaction_kapp_change_file = "./analysis/reaction_kapp_change_c13_enzuse.csv"
kcat_data_colect_file="./analysis/reaction_max_df.csv"

change_reaction_list_round1=change_reaction_kcat_by_database_kapp(select_reaction,kcat_data_colect_file,reaction_kcat_mw_file,reaction_kapp_change_file)
print(change_reaction_list_round1)

['PDH', 'NADH16pp', 'PSERT', 'ATPS4rpp', 'GND', 'FBA', 'PAPSR2', 'KARA2', 'TKT2', 'ENO', 'GAPD', 'ACGS', 'PPC', 'PGL', 'SADT2', 'CS', 'ADSS', 'IPPMIa_reverse', 'IPPMIb_reverse', 'AKGDH', 'CYTBO3_4pp', 'ACCOAC']
['NADH16pp', 'PSERT', 'GND', 'FBA', 'PAPSR2', 'KARA2', 'TKT2', 'ENO', 'ACGS', 'PPC', 'SADT2', 'CS', 'ADSS', 'IPPMIa_reverse', 'IPPMIb_reverse', 'AKGDH', 'CYTBO3_4pp', 'ACCOAC']


In [24]:
model_file = './data/iML1515.xml' 
reaction_kcat_mw_file="./analysis/reaction_kapp_change_c13_enzuse.csv"
json_output_file="./model/iML1515_irr_enz_constraint_adj_round3.json"
f = 0.406
ptot = 0.56 
sigma = 1
lowerbound = 0   
upperbound = round(ptot * f * sigma, 3)

trans_model2enz_json_model(model_file, reaction_kcat_mw_file, f, ptot, sigma, lowerbound, upperbound, json_output_file)

In [25]:
json_model_path = "./model/iML1515_irr_enz_constraint_adj_round3.json"
fluxes_outfile = './analysis/ECMpy_kapp_adj_round3_solution_df_pfba.csv'
reaction_kcat_mw_file="./analysis/reaction_kapp_change_c13_enzuse.csv"
enz_model=get_enzyme_constraint_model(json_model_path)
enz_model_pfba_solution = get_fluxes_detail_in_model(enz_model,fluxes_outfile,reaction_kcat_mw_file)
print(enz_model_pfba_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M'])

0.8762755722525362


In [26]:
c13reaction_file = './data/C13reaction.csv' 
c13reaction = pd.read_csv(c13reaction_file, index_col=0)
c13reaction = list(c13reaction.index)

enz_model_pfba_solution_select = enz_model_pfba_solution[enz_model_pfba_solution['fluxes']>0]
enz_model_pfba_solution_select_id = list(enz_model_pfba_solution_select.index)
c13reaction_2_enz_model_diff=list(set(c13reaction).difference(set(enz_model_pfba_solution_select_id)))
print (c13reaction_2_enz_model_diff)

['ACKr_reverse', 'PFL', 'ICL', 'MALS', 'ME2', 'PTAr']


In [45]:
reaction_kcat_mw_file = "./analysis/reaction_kapp_change_c13_enzuse.csv"
reaction_kapp_change_file = "./analysis/reaction_kapp_change_c13_enzuse_manual.csv"
select_reaction=['PDH','CS','ACKr_reverse','PTAr']
#c13reaction_selecet=['CS','ACONTa','ACONTb','ICDHyr','MALS', 'MDH', 'ICL', 'SUCOAS_reverse', 'SUCDi', 'AKGDH']
change_fold=[1000,0.03,100,100]
change_reaction_list_round1=change_reaction_kcat_by_foldlist(select_reaction,change_fold,reaction_kcat_mw_file,reaction_kapp_change_file)
print(change_reaction_list_round1)

model_file = './data/iML1515.xml' 
reaction_kcat_mw_file="./analysis/reaction_kapp_change_c13_enzuse_manual.csv"
json_output_file="./model/iML1515_irr_enz_constraint_adj_manual.json"
#The enzyme mass fraction 
f = 0.406
# The total protein fraction in cell.
ptot = 0.56 
# The approximated average saturation of enzyme.
sigma = 1 
# Lowerbound  of enzyme concentration constraint. 
lowerbound = 0   
upperbound = round(ptot * f * sigma, 3)

trans_model2enz_json_model(model_file, reaction_kcat_mw_file, f, ptot, sigma, lowerbound, upperbound, json_output_file)

json_model_path = "./model/iML1515_irr_enz_constraint_adj_manual.json"
fluxes_outfile = './analysis/ECMpy_kapp_adj_manual_solution_df_pfba.csv'
reaction_kcat_mw_file="./analysis/reaction_kapp_change_c13_enzuse_manual.csv"
enz_model=get_enzyme_constraint_model(json_model_path)
#rglcex=enz_model.reactions.get_by_id("MDH2")
#rglcex.bounds=(0, 0)

enz_model_pfba_solution = get_fluxes_detail_in_model(enz_model,fluxes_outfile,reaction_kcat_mw_file)
print(enz_model_pfba_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M'])

c13reaction_file = './data/C13reaction.csv' 
c13reaction = pd.read_csv(c13reaction_file, index_col=0)
c13reaction = list(c13reaction.index)

enz_model_pfba_solution_select = enz_model_pfba_solution[enz_model_pfba_solution['fluxes']>0]
enz_model_pfba_solution_select_id = list(enz_model_pfba_solution_select.index)
c13reaction_2_enz_model_diff=list(set(c13reaction).difference(set(enz_model_pfba_solution_select_id)))
print (c13reaction_2_enz_model_diff)

['PDH', 'CS', 'ACKr_reverse', 'PTAr']
0.8764409655911223
['ACKr_reverse', 'PFL', 'ICL', 'MALS', 'ME2', 'PTAr']


In [27]:
#葡糖糖浓度增加
json_model_path = "./model/iML1515_irr_enz_constraint_adj_manual.json"
fluxes_outfile = './analysis/ECMpy_kapp_adj_manual_glc_solution_df_pfba.csv'
reaction_kcat_mw_file="./analysis/reaction_kapp_change_c13_enzuse.csv"
enz_model=get_enzyme_constraint_model(json_model_path)
rglcex=enz_model.reactions.get_by_id("EX_glc__D_e_reverse")
rglcex.bounds=(0, 50)

enz_model_pfba_solution = get_fluxes_detail_in_model(enz_model,fluxes_outfile,reaction_kcat_mw_file)
print(enz_model_pfba_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M'])

c13reaction_file = './data/C13reaction.csv' 
c13reaction = pd.read_csv(c13reaction_file, index_col=0)
c13reaction = list(c13reaction.index)

enz_model_pfba_solution_select = enz_model_pfba_solution[enz_model_pfba_solution['fluxes']>0]
enz_model_pfba_solution_select_id = list(enz_model_pfba_solution_select.index)
c13reaction_2_enz_model_diff=list(set(c13reaction).difference(set(enz_model_pfba_solution_select_id)))
print (c13reaction_2_enz_model_diff)

1.0074240822023213
['ME2', 'ACKr_reverse', 'PTAr', 'ICL', 'TKT2', 'PDH']


# Step4: Solveing enzyme concentration constraint by COBRApy.

In [21]:
#run enzyme constraint metabolic model
json_model_path="./model/iML1515_irr_enz_constraint_adj_round2.json"
enz_model=get_enzyme_constraint_model(json_model_path)
pfba_solution = cobra.flux_analysis.pfba(enz_model)
pfba_solution_df = pfba_solution.to_frame()
#pfba_solution_df.to_csv('./analysis/ECMpy_solution_df_pfba.csv')
pfba_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M']

0.7901116510198356

In [22]:
#run genome-scale metabolic model
norm_model=cobra.io.json.load_json_model(json_model_path)
pfba_solution = cobra.flux_analysis.pfba(norm_model)
pfba_solution_df = pfba_solution.to_frame()
#pfba_solution_df.to_csv('./analysis/Orimodel_solution_df_pfba.csv')
pfba_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M']


0.8769972144269698